<a href="https://colab.research.google.com/github/alexdibol/ML_FINANCE_TRAINING/blob/master/Copia_de_4_3_2_LSTM_WITH_KERAS_AND_TB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INTRODUCTION TO LSTM MODELS WITH KERAS

http://hunterheidenreich.com/blog/intro-to-lstms-in-keras/

https://youtu.be/Sqx2dmlJN6I


## 1. THE DATASET

In [1]:
from keras.datasets import imdb
max_words=20000
(x_train,y_train),(x_test,y_test)=imdb.load_data(num_words=max_words)

Using TensorFlow backend.


17465344/17464789 [==============================] - 0s 0us/step


In [2]:
print("X_train length: ", len(x_train))
print("X_test length: ", len(x_test))

X_train length:  25000
X_test length:  25000


## 2. MAPPING WORDS TO INDEX

In [3]:
word_to_index = imdb.get_word_index()
index_to_word = {v: k for k, v in word_to_index.items()}

1646592/1641221 [==============================] - 0s 0us/step


AN EXAMPLE OF HOW PHRASES ARE LINKED TO WORDS

In [4]:
print(x_train[0])
print(" ".join([index_to_word[x] for x in x_train[0]]))

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
the as you with out themselves powerful lets lov

THE OUTPUT VALUE IS BINARY. EITHER RECOMMEND OR NOT THE MOVIE

In [5]:
print("Min value:", min(y_train), "Max value:", max(y_train))

Min value: 0 Max value: 1


OTHER IMPORTANT METRICS

In [8]:
import numpy as np

average_length = np.mean([len(x) for x in x_train])
median_length = sorted([len(x) for x in x_train])[len(x_train) // 2]

print("Average sequence length: ", average_length)
print("Median sequence length: ", median_length)

max_sequence_length = 180

Average sequence length:  180.0
Median sequence length:  180


## 3. PREPROCESSING

In [9]:
from keras.preprocessing import sequence

x_train = sequence.pad_sequences(x_train, maxlen=max_sequence_length, padding='post', truncating='post')
x_test = sequence.pad_sequences(x_test, maxlen=max_sequence_length, padding='post', truncating='post')

print('X_train shape: ', x_train.shape)

X_train shape:  (25000, 180)


## 4. LOADING KERAS

In [16]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dense
import keras
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x

except Exception:
    pass
import tensorflow as tf

# Load the TensorBoard notebook extension.
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [0]:
import os
from tensorboard.plugins import projector

In [0]:
LOGDIR='content/logs/imdb-LSTM/'
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=LOGDIR)
file_writer = tf.summary.create_file_writer(LOGDIR)

## 5. DEFINING A SINGLE LAYER LSTM MODEL

In [0]:
# Single layer LSTM example

hidden_size = 32

sl_model = Sequential()
sl_model.add(Embedding(max_words, hidden_size))
sl_model.add(LSTM(hidden_size, activation='tanh', dropout=0.2, recurrent_dropout=0.2))
sl_model.add(Dense(1, activation='sigmoid'))


In [29]:
sl_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 32)          640000    
_________________________________________________________________
lstm_5 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 648,353
Trainable params: 648,353
Non-trainable params: 0
_________________________________________________________________


In [0]:
sl_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

epochs = 3

sl_model.fit(x_train, y_train, epochs=epochs, shuffle=True,callbacks=[tensorboard_callback] )
loss, acc = sl_model.evaluate(x_test, y_test)

## 6. DEFINING A TWO-LAYER LSTM MODEL

In [7]:
d_model = Sequential()
d_model.add(Embedding(max_words, hidden_size))
d_model.add(LSTM(hidden_size, activation='tanh', dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
d_model.add(LSTM(hidden_size, activation='tanh', dropout=0.2, recurrent_dropout=0.2))
d_model.add(Dense(1, activation='sigmoid'))
d_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

d_model.fit(x_train, y_train, epochs=epochs, shuffle=True)
d_loss, d_acc = d_model.evaluate(x_test, y_test)

print('Single layer model -- ACC {} -- LOSS {}'.format(acc, loss))
print('Double layer model -- ACC {} -- LOSS {}'.format(d_acc, d_loss))

X_train shape:  (25000, 180)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3
10688/25000 [===========>..................] - ETA: 50s - loss: 0.6908 - accuracy: 0.5221

KeyboardInterrupt: ignored